## Dispersion parameters
This notebook illustrates the role of the dispersion parameters included in the calibration algorithm.

In [ ]:
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from emutools.utils import load_param_info
from aust_covid.inputs import get_ifrs
import arviz as az
from emutools.tex import DummyTexDoc
from aust_covid.constants import RUN_IDS, PRIMARY_ANALYSIS, BURN_IN, set_project_base_path
import numpy as np
from aust_covid.plotting import plot_dispersion_examples

project_paths = set_project_base_path("../")
SUPPLEMENT_PATH = project_paths["SUPPLEMENT_PATH"]
RUNS_PATH = project_paths["RUNS_PATH"]

In [ ]:
dummy_doc = DummyTexDoc()
param_info = load_param_info()
epi_model = build_model(dummy_doc, param_info["abbreviations"], mobility_ext=True)
targets = get_targets(dummy_doc)
prior_names = [p.name for p in get_priors(False, param_info["abbreviations"], dummy_doc)]
parameters = param_info["value"].to_dict() | get_ifrs(dummy_doc)
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / "output/calib_full_out.nc")
idata = idata.sel(draw=np.s_[BURN_IN:])

In [ ]:
plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, ["10, 10, 100", "100, 10, 10"], np.linspace(0.1, 0.9, 9))